In [158]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_neo4j import Neo4jVector
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain_core.documents import Document
from langchain_neo4j import Neo4jGraph
from langchain_neo4j.graphs.graph_document import Node, Relationship,GraphDocument
from dotenv import load_dotenv
import os
import re

In [159]:
load_dotenv()
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME= os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD= os.getenv("NEO4J_PASSWORD")
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
OPENROUTER_MODEL_NAME = os.getenv("OPENROUTER_MODEL_NAME")

In [160]:
graph = Neo4jGraph( url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)

In [161]:
law = Node(id="rgpd",type="Loi",properties={
        "nom":"RGPD",
        "objectif":"la protection des personnes physiques à l'égard du traitement des données à caractère personnel et à la libre circulation de ces données",
        "organisme":"Parlement européen",
        "date_application":"25 mai 2018",
        "date_publication":"27 avril 2016"
    })

recital = Node(id="rgpd-considerant", type="considerant")
relationship = Relationship(source=law, target=recital,type="A_CONSIDERANT")

In [162]:
graph_doc = GraphDocument(nodes=[law,recital], relationships=[relationship])

In [163]:
loader = UnstructuredHTMLLoader("../../data/rgpd.html")
data = loader.load()

In [164]:
recitals_text, law_text = re.split(
    r"\nCHAPITRE\s+I\b",
    data[0].page_content,
    maxsplit=1
)


In [165]:
recital_blocks = re.split(r"\n\((\d+)\)\s*", recitals_text)
recital_blocks

["4.5.2016 FR Journal officiel de l'Union européenne L 119/1\n\nRÈGLEMENT (UE) 2016/679 DU PARLEMENT EUROPÉEN ET DU CONSEIL\n\ndu 27 avril 2016\n\nrelatif à la protection des personnes physiques à l'égard du traitement des données à caractère personnel et à la libre circulation de ces données, et abrogeant la directive 95/46/CE (règlement général sur la protection des données)\n\n(Texte présentant de l'intérêt pour l'EEE)\n\nLE PARLEMENT EUROPÉEN ET LE CONSEIL DE L'UNION EUROPÉENNE,\n\nvu le traité sur le fonctionnement de l'Union européenne, et notamment son article 16,\n\nvu la proposition de la Commission européenne,\n\naprès transmission du projet d'acte législatif aux parlements nationaux,\n\nvu l'avis du Comité économique et social européen (1),\n\nvu l'avis du Comité des régions (2),\n\nstatuant conformément à la procédure législative ordinaire (3),\n\nconsidérant ce qui suit:\n",
 '1',
 "La protection des personnes physiques à l'égard du traitement des données à caractère perso

In [166]:
print(len(recital_blocks))

347


In [167]:

for i in range(1, len(recital_blocks), 2):
    recital_number = recital_blocks[i]
    recital_text = recital_blocks[i + 1].strip()
    recital_node = Node(
        id=f"considerant-{recital_number}",
        type="articleConsiderant",
        properties={
            "text":recital_text,
            "loi":"RGPD",
            "partie":"considerant"
        }
    )
    graph_doc.nodes.append(recital_node)
    graph_doc.relationships.extend([
        Relationship(source=recital,target=recital_node,type="A_ARTICLE")
    ])


In [168]:
print(law_text)



Dispositions générales

Article premier

Objet et objectifs

1. Le présent règlement établit des règles relatives à la protection des personnes physiques à l'égard du traitement des données à caractère personnel et des règles relatives à la libre circulation de ces données.

2. Le présent règlement protège les libertés et droits fondamentaux des personnes physiques, et en particulier leur droit à la protection des données à caractère personnel.

3. La libre circulation des données à caractère personnel au sein de l'Union n'est ni limitée ni interdite pour des motifs liés à la protection des personnes physiques à l'égard du traitement des données à caractère personnel.

Article 2

Champ d'application matériel

1. Le présent règlement s'applique au traitement de données à caractère personnel, automatisé en tout ou en partie, ainsi qu'au traitement non automatisé de données à caractère personnel contenues ou appelées à figurer dans un fichier.

2. Le présent règlement ne s'applique pas 

In [169]:
chapter_blocks = re.split(
    r"\nCHAPITRE\s+([IVXLC]+)\b\s*(.*)",
    law_text
)

In [170]:
chapter_blocks.insert(0,"I")

In [171]:
chapter_blocks.insert(1,"Dispositions générales")

In [172]:
def split_sections(chapter_text):
    sections = re.split(
        r"\n\nSection\s+(\d+)\n\n*(.*)",
        chapter_text
    )
    return sections



In [173]:
def split_articles(section_text):
    articles = re.split(
        r"\n\nArticle\s+(\d+)\b\s*(.*)",
        section_text
    )
    return articles

In [174]:
for i in range(0,len(chapter_blocks), 3):
    chapter_number = chapter_blocks[i]
    chapter_title = chapter_blocks[i + 1].strip()
    sections = split_sections(chapter_blocks[i+2])[1:]
    chapter_node = Node(
        id=f"chapitre-{chapter_number}",
        type="chapitre",
        properties={
            "titre":chapter_title,
            "loi":"RGPD",
            "partie":"chapitre",
            "numero":chapter_number
        }
    )

    graph_doc.nodes.append(chapter_node)
    graph_doc.relationships.extend([
        Relationship(source=law,target=chapter_node,type="A_CHAPITRE")
    ])

    if len(sections) >= 3 :
        for j in range(0,len(sections),3):
            section_node = Node(
                id=f"section-{chapter_number}-{sections[j]}",
                type="section",
                properties={
                    "titre":sections[j+1],
                    "loi":"RGPD",
                    "chapitre":chapter_number,
                    "partie":"section"
                }                
            )

            graph_doc.nodes.append(section_node)
            graph_doc.relationships.extend([
                Relationship(source=chapter_node,target=section_node,type="A_SECTION")
            ])
            
            articles = split_articles(sections[j+2])[1:]
            for k in range(0,len(articles),3):
                article_node = Node(
                    id=f"article-{articles[k]}",
                    type="article",
                    properties={
                        "titre":articles[k+1],
                        "text":articles[k+2],
                        "loi":"RGPD",
                        "section":sections[j],
                        "chapitre":chapter_number,
                        "article":articles[k],
                        "partie":"article"
                    }
                )
                graph_doc.nodes.append(article_node)
                graph_doc.relationships.extend([
                    Relationship(source=section_node,target=article_node,type="A_ARTICLE")
                ])

    else:
        articles = split_articles(chapter_blocks[i+2])[1:]
        for k in range(0,len(articles),3):
            article_node = Node(
                    id=f"article-{articles[k]}",
                    type="article",
                    properties={
                        "titre":articles[k+1],
                        "text":articles[k+2],
                        "loi":"RGPD",
                        "chapitre":chapter_number,
                        "article":articles[k],
                        "partie":"article"
                    }
            )
            graph_doc.nodes.append(article_node)
            graph_doc.relationships.extend([
                    Relationship(source=chapter_node,target=article_node,type="A_ARTICLE")
            ])
    


In [175]:
graph.refresh_schema()
graph.add_graph_documents([graph_doc])

In [198]:
llm = ChatOpenAI(
    api_key=OPENROUTER_API_KEY,
    base_url="https://openrouter.ai/api/v1",
    model=OPENROUTER_MODEL_NAME,
    default_headers={
        "HTTP-Referer": "http://localhost",   
        "X-Title": "LegalGraphRAG",            
    },

)

In [199]:
prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(""" 
		You are an expert Neo4j developer. Use the following database schema to write a Cypher statement to answer the user's question. Only generate the Cypher statement, no pre-amble. Do not return any Markdown.
	Schema: 
    {schema}
    
	Question: {input}""", 
    partial_variables={"schema": graph.schema})
])

In [200]:
text_to_cypher_chain = prompt | llm | StrOutputParser()

In [201]:
graphrag_qa_prompt = ChatPromptTemplate.from_messages([  
        SystemMessagePromptTemplate.from_template("""    You are a helpful legal assistant answering questions about General Data Protection Regulation GDPR the European Union regulation on information privacy in the European Union (EU law in french.    
                                                You are given a question and a context.    
                                                Question: {input}    
                                                """
                                                ),
        SystemMessagePromptTemplate.from_template("""    The following context has been retrieved from the database using vector search to help    
                                                  you answer the question:     
                                                  {vectors}   
                                                   """),
        SystemMessagePromptTemplate.from_template("""    The following data has been retrieved from the knowledge graph using Cypher to    
                                                   answer the question.      
                                                  {records}    
                                                  You can treat any information contained from the knowledge graph as authoritative.    
                                                  If the information does not exist in this answer, fall back to vector search results.    
                                                  If the answer is not included in either just say that you don't know and don't rely    
                                                  on pre-existing knowledge."""),
])


In [195]:
store = Neo4jVector.from_existing_graph( HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"),    
                                        url=NEO4J_URI,    
                                        username=NEO4J_USERNAME,    
                                        password=NEO4J_PASSWORD,    
                                        node_label="article",    
                                        text_node_properties=["text"],    
                                        embedding_node_property="embedding",    
                                        index_name="articles",
                                        )
retriever = store.as_retriever()

In [202]:
graphrag_qa_chain = RunnablePassthrough.assign(    
    vectors=RunnableLambda(lambda x: retriever.invoke(x["input"])),    
    records=text_to_cypher_chain | graph.query) | graphrag_qa_prompt    | llm    | StrOutputParser()


In [205]:
graphrag_qa_chain.invoke({"input":"Qu’est-ce qu’une donnée à caractère personnel ?"})


"Le RGPD (Règlement Général sur la Protection des Données) définit les données à caractère personnel comme toute information se rapportant à une personne physique identifiée ou identifiable. Une personne physique identifiable est une personne qui peut être identifiée, directement ou indirectement, notamment par référence à un identifiant, tel qu'un nom, un numéro d'identification, des données de localisation, un identifiant en ligne, ou à un ou plusieurs éléments spécifiques propres à son identité physique, physiologique, génétique, psychique, économique, culturelle ou sociale.\n\nCette définition est fondamentale pour comprendre l'application du RGPD, car elle détermine quelles informations sont protégées par ce règlement."

In [206]:
graphrag_qa_chain.invoke({"input":"Qui est le responsable du traitement selon le RGPD ?"})

"Selon le RGPD, le responsable du traitement est défini comme la personne physique ou morale, l'autorité publique, le service ou un autre organisme qui, seul ou conjointement avec d'autres, détermine les finalités et les moyens du traitement des données à caractère personnel. Cette définition est donnée à l'article 4, paragraphe 7, du RGPD."

In [207]:
graphrag_qa_chain.invoke({"input":"Dans quels cas le consentement est-il requis ?"})

"Le consentement est requis dans plusieurs cas spécifiques selon le RGPD. Voici les principaux cas où le consentement est nécessaire :\n\n1. **Traitement des données à caractère personnel** : Le consentement est requis lorsque le traitement des données à caractère personnel est basé sur le consentement de la personne concernée pour une ou plusieurs finalités spécifiques (Article 6, paragraphe 1, point a)).\n\n2. **Traitement des catégories particulières de données** : Pour le traitement des données sensibles (comme les données révélant l'origine raciale ou ethnique, les opinions politiques, les convictions religieuses ou philosophiques, l'appartenance syndicale, les données génétiques, biométriques, concernant la santé ou la vie sexuelle), le consentement explicite de la personne concernée est généralement requis, sauf dans certaines exceptions prévues par la loi (Article 9, paragraphe 2, point a)).\n\n3. **Services de la société de l'information pour les enfants** : Pour l'offre direc

In [208]:
graphrag_qa_chain.invoke({"input":"Quels sont les principes du traitement des données personnelles ?"})

"Les principes du traitement des données personnelles selon le RGPD (Règlement Général sur la Protection des Données) sont énoncés dans plusieurs articles, notamment les articles 5, 6 et 9 du RGPD. Bien que les articles fournis dans le contexte ne les détaillent pas explicitement, voici les principes fondamentaux du traitement des données personnelles tels que définis par le RGPD :\n\n1. **Licéité, loyauté et transparence** : Les données doivent être traitées de manière licite, loyale et transparente pour la personne concernée.\n2. **Limitation des finalités** : Les données doivent être collectées pour des finalités déterminées, explicites et légitimes, et ne pas être traitées ultérieurement de manière incompatible avec ces finalités.\n3. **Minimisation des données** : Les données collectées doivent être adéquates, pertinentes et limitées à ce qui est nécessaire au regard des finalités pour lesquelles elles sont traitées.\n4. **Exactitude** : Les données doivent être exactes et, si néc

Article -[:IMPOSES]-> Obligation

Obligation -[:APPLIES_TO]-> ActorRole

Obligation -[:REQUIRES]-> Measure

Obligation -[:SUBJECT_TO]-> Condition

Obligation -[:HAS_EXCEPTION]-> Exception

Concept -[:MENTIONED_IN]-> Article